In [114]:
#from mlxtend.frequent_patterns import apriori
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [115]:
from scipy.io import arff
data, meta = arff.loadarff('./data/supermarket.arff')

data

array([(b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b't', b't', b'?', b't', b'?', b't', b'?', b'?', b't', b'?', b'?', b'?', b't', b't', b't', b't', b'?', b't', b'?', b't', b't', b'?', b'?', b'?', b'?', b'?', b'?', b't', b't', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b't', b'?', b'?', b't', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'

In [116]:
supermarket_one_hot = pd.get_dummies(pd.DataFrame(data))
# find cols with interrogation marks
cols_with_interrogation_mark = supermarket_one_hot.filter(regex='\'\?\'$',axis=1).columns # Fix regex as it did not match any cols on my environment...
# delete columsn with interrogation marks in it
supermarket_one_hot.drop(cols_with_interrogation_mark,axis=1,inplace=True)

In [117]:
from mlxtend.frequent_patterns import apriori
apriori_result = apriori(supermarket_one_hot, min_support=0.1)
print('apriori_result n of rows : {}'.format(apriori_result.shape[0]))

apriori_result n of rows : 10282


In [118]:
# generate a bar chart, itemsets frequencies in terms of items containes
import matplotlib
import matplotlib.pyplot as plt

max_itemsets_len = apriori_result.itemsets.map(len).max()
frequencies = {}

for i in range(1, max_itemsets_len + 1):
   frequencies[str(i)] = apriori_result.loc[map(lambda x: len(x) == i, apriori_result['itemsets'])].shape[0]

x = np.arange(len(frequencies.keys()))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()

rects = ax.bar(x - width/2, frequencies.values(), width, label='itemsets')

# Add value to each bar
for rect in rects:
    height = rect.get_height()
    ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequency')
ax.set_xlabel('Items contained')
ax.set_title('itemsets frequencies')
ax.set_xticks(x)
ax.set_xticklabels(frequencies.keys())
ax.legend()
plt.savefig('fig/itemsets_frequencies')
plt.close(fig)

In [119]:
# build association rules
from mlxtend.frequent_patterns import association_rules
min_threshold = 0.7
dataset_association_rules = association_rules(apriori_result, min_threshold=min_threshold)
print('Number of association rules : {}'.format(dataset_association_rules.shape[0]))
print('-------------')
print('First association rule : ')
print(dataset_association_rules.iloc[0])

Number of association rules : 24570
-------------
First association rule : 
antecedents                  (0)
consequents                 (11)
antecedent support      0.226281
consequent support      0.719689
support                 0.171601
confidence              0.758357
lift                     1.05373
leverage              0.00874991
conviction               1.16002
Name: 0, dtype: object


In [120]:
# rules with 5 items (4 antecedents, 1 consequent)

# select those with 4 antecedents
rules_with_5_items = dataset_association_rules.loc[map(lambda x: len(x) == 4, dataset_association_rules['antecedents'])]
rules_with_5_items = rules_with_5_items.loc[map(lambda x: len(x) == 1, rules_with_5_items['consequents'])]

print('Number of rules with 5 items : {}'.format(rules_with_5_items.shape[0]))

Number of rules with 5 items : 8811


In [121]:
rules_with_5_items.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000,8811.000000
mean,0.148910,0.603212,0.117313,0.791140,1.332255,0.027822,1.978909
std,0.022173,0.082035,0.015988,0.054682,0.177603,0.009443,0.413518
min,0.108494,0.362870,0.100065,0.700000,1.062178,0.006364,1.189966
25%,0.133348,0.563000,0.105252,0.747244,1.220572,0.020979,1.675085
50%,0.144586,0.604063,0.112816,0.783489,1.290764,0.026576,1.930478
75%,0.159282,0.640156,0.124919,0.829582,1.388577,0.032749,2.231455
max,0.283337,0.719689,0.202939,0.929795,2.199346,0.072145,4.517902


In [122]:
# find best rules for each metrics
metrics = ('confidence', 'lift', 'leverage', 'conviction')

for metric in metrics:
    print('--------------------------------------------------------------------------')
    print('Best association rule for metric {}'.format(metric))
    print('-----------------------------------------------------')
    print(dataset_association_rules.iloc[dataset_association_rules[metric].idxmax()])
    print('--------------------------------------------------------------------------')


--------------------------------------------------------------------------
Best association rule for metric confidence
-----------------------------------------------------
antecedents           (37, 77, 15, 80, 122, 29)
consequents                                (11)
antecedent support                     0.110223
consequent support                     0.719689
support                                0.103307
confidence                             0.937255
lift                                    1.30231
leverage                              0.0239807
conviction                              4.46746
Name: 24524, dtype: object
--------------------------------------------------------------------------
--------------------------------------------------------------------------
Best association rule for metric lift
-----------------------------------------------------
antecedents           (38, 77, 15, 80, 29)
consequents                      (122, 11)
antecedent support                0.1432

In [123]:
# clean columns names
import re
supermarket_one_hot.rename(columns=lambda x: re.sub('_b\'t\'$', '', x), inplace=True)

In [129]:
# search for frequent items bought with coffee and milk-cream
positioning_apriori_result = apriori(supermarket_one_hot, min_support=0.1, use_colnames=True)
itemsets_with_coffee_and_milkcream = positioning_apriori_result.loc[map(lambda x: 'milk-cream' in x and 'coffee' in x, positioning_apriori_result['itemsets'])]

frequently_bought_items = []
excluded = ('coffee', 'milk-cream')
for item in itemsets_with_coffee_and_milkcream['itemsets']:
    not_coffee_nor_milkcream = [element for element in item if element not in excluded]

    if len(not_coffee_nor_milkcream):
        frequently_bought_items.extend(not_coffee_nor_milkcream)


frequently_bought_items = list(dict.fromkeys(frequently_bought_items)) # remove duplicates

print(frequently_bought_items)

['bread and cake', 'baking needs', 'biscuits', 'frozen foods', 'margarine', 'fruit', 'vegetables']
